In [1]:
import numpy as np
import pandas as pd
from dask import dataframe as dd
import os
import plotly.express as px

In [39]:
update_file = "/home/juuso/Documents/gradu/ADAUSDT_T_DEPTH_2021-10-30/ADAUSDT_T_DEPTH_2021-10-30_depth_update.csv"
snap_file = "/home/juuso/Documents/gradu/ADAUSDT_T_DEPTH_2021-10-30/ADAUSDT_T_DEPTH_2021-10-30_depth_snap.csv"
res_file = "/home/juuso/Documents/gradu/ADAUSDT_T_DEPTH_2021-10-30/parsed.csv"
parsed = "/home/juuso/Documents/gradu/parsed_data/2021-12-22_parsed.csv"
parsed_path = "/home/juuso/Documents/gradu/parsed_data"
hb_logs = "/home/juuso/Documents/hummingbot/local_v2/hummingbot/logs/intensity_sample.csv"
as_logs = "/home/juuso/Documents/gradu/logs/as_env_sample_1675594208.log"

In [43]:
data_lines = []
with open(as_logs, "r") as f:
    while True:
        line = f.readline()
        if line == "":
            break
        line = line.rstrip().split(":")
        data = line[2].split(",")
        data_lines.append(data)
df = pd.DataFrame(data_lines)
update_df = df[df[0] == "update"]
action_df = df[df[0] == "action"]
# action_df.drop(columns=[8,9], inplace=True)


action_df.columns = ["type", "timestamp", "action", "price", "amount", "mid_price", 'base',"quote"]
action_df['value'] = action_df.base.astype(float) * action_df.mid_price.astype(float) + action_df.quote.astype(float)
action_df.action.value_counts()


market_sell    1211
market_buy      151
Name: action, dtype: int64

In [9]:

update_df.columns = ["type", "timestamp", "bid_price","ask_price","mid_price", "vol", "intensity", "inventory", "reservation_price", "spread"]
update_df


,type,timestamp,bid_price,ask_price,mid_price,vol,intensity,inventory,reservation_price,spread
0,update,1640047248223.0,0,0,1.2355,0.00033139453077169975,4927.856397845143,0.0,1.2355,0.0004389913176618919
1,update,1640047248237.0,1.2353,1.2357,1.2355,0.00033139453077169975,4927.856397845143,0.0,1.2355,0.0004389913176618919
3,update,1640047258339.0,1.2353,1000000000,1.2355,0.00033139453077169975,4927.856397845143,-0.0012354997529000495,1.2355000409437862,0.0004389913176618919
4,update,1640047268422.0,1.2353,1.2357,1.2355,0.00033139453077169975,4927.856397845143,-0.0012354997529000495,1.2355000409437862,0.0004389913176618919
6,update,1640047278480.0,0,1.2357,1.2351,0.00033139453077169975,4927.856397845143,0.0,1.2351,0.0004389913176618919
...,...,...,...,...,...,...,...,...,...,...
40606,update,1640217552668.0,0,1000000000,1.3264,0.00040069132869671785,48519.45073935687,-0.02773309544054292,1.326401111241086,8.12896724367964e-05
40609,update,1640217562747.0,0,1000000000,1.327,0.00040069132869671785,48519.45073935687,-0.027745988645123017,1.3270011117577056,8.12896724367964e-05
40611,update,1640217572816.0,1.327,1000000000,1.3276,0.00040069132869671785,48519.45073935687,-0.029080751078161785,1.3276011652404789,8.12896724367964e-05
40614,update,1640217582912.0,0,1000000000,1.3272,0.00040069132869671785,48519.45073935687,-0.02907173445298818,1.3272011648791904,8.12896724367964e-05


In [ ]:
px.line(update_df[::100], x="timestamp", y=["bid_price", "ask_price", "mid_price"])

In [ ]:
px.line(update_df[:100], x="timestamp", y=["mid_price", "reservation_price"])

In [26]:

# action_df.columns = ["type", "timestamp", "action", "price", "amount", "mid_price", 'base',"quote"]
action_df['value'] = action_df.base.astype(float) * action_df.mid_price.astype(float) + action_df.quote.astype(float)

px.line(action_df[::100], x="timestamp", y=["value"])
volume = (action_df.amount.astype(float) * action_df.price.astype(float)).sum()
end_total = (action_df.quote.astype(float) + action_df.base.astype(float) * action_df.mid_price.astype(float)).values[-1]
start_total = (action_df.quote.astype(float) + action_df.base.astype(float) * action_df.mid_price.astype(float)).values[0]

profit = end_total - start_total
break_even_fees = profit / volume
print(f"Break even fees: {break_even_fees}")
end_total

Break even fees: 0.0001423536675286002


/tmp/ipykernel_18120/2354574168.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



1004.3559000000052

In [ ]:
px.line(action_df[::100], x="timestamp", y=["base"])

In [ ]:
for chunk in pd.read_csv(update_file, chunksize=1_000_000):
    ts_chunk = chunk[chunk.timestamp == 1635552439148]
    if ts_chunk.empty:
        continue
    else:
        print('found')
        break

ts_chunk

In [ ]:
df = pd.read_csv("/home/juuso/Documents/gradu/parsed_data/AvellanedaStoikov/data.csv", nrows=1_000_000)
df = df.rename(columns={'size':"trade_size"})
df = df[df.trade_size != 0]

In [ ]:
df['price_diff'] = np.abs(df.price - df.mid_price)
# choose every 10th row
part = df.iloc[::10]

px.scatter(part, x='price_diff', y='trade_size', trendline='ols')
# df

In [ ]:
grouped = df.groupby('price_diff').sum().reset_index()

px.scatter(grouped, x='price_diff', y='trade_size', trendline='ols')

In [45]:
def abc(a, b, c, **kwargs):
    return a + b + c

dic = {'a':1, 'b':2, 'c':3, 'd':4}
abc(**dic)

TypeError: abc() got an unexpected keyword argument 'd'